<a href="https://colab.research.google.com/github/hoangvn111/Data-Cleaning-in-Python-Advanced/blob/master/Data_Cleaning_in_Python_Advanced.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Introduction

The dataset we will be working with is based off this CSV of Hacker News stories from September 2015 to September 2016. The columns in the dataset are explained below:

* id: The unique identifier from Hacker News for the story
* title: The title of the story
* url: The URL that the stories links to, if the story has a URL
* num_points: The number of points the story acquired, calculated as the total number of upvotes minus the total number of downvotes
* num_comments: The number of comments that were made on the story
* author: The username of the person who submitted the story
* created_at: The date and time at which the story was submitted

In [2]:
import pandas as pd 
hn = pd.read_csv('/content/drive/MyDrive/my_datasets/Data Cleaning in Python Advanced/hacker_news.csv')

In [3]:
hn.head()

,id,title,url,num_points,num_comments,author,created_at
0,12224879,Interactive Dynamic Video,http://www.interactivedynamicvideo.com/,386,52,ne0phyte,8/4/2016 11:52
1,11964716,Florida DJs May Face Felony for April Fools' W...,http://www.thewire.com/entertainment/2013/04/f...,2,1,vezycash,6/23/2016 22:20
2,11919867,Technology ventures: From Idea to Enterprise,https://www.amazon.com/Technology-Ventures-Ent...,3,1,hswarna,6/17/2016 0:01
3,10301696,Note by Note: The Making of Steinway L1037 (2007),http://www.nytimes.com/2007/11/07/movies/07ste...,8,2,walterbell,9/30/2015 4:12
4,10482257,Title II kills investment? Comcast and other I...,http://arstechnica.com/business/2015/10/comcas...,53,22,Deinos,10/31/2015 9:48


In [4]:
hn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20099 entries, 0 to 20098
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            20099 non-null  int64 
 1   title         20099 non-null  object
 2   url           17659 non-null  object
 3   num_points    20099 non-null  int64 
 4   num_comments  20099 non-null  int64 
 5   author        20099 non-null  object
 6   created_at    20099 non-null  object
dtypes: int64(3), object(4)
memory usage: 1.1+ MB


## Regular Expression Basics

### The Regular Expression Module

We're going to use this technique to find out how many times Python is mentioned in the title of stories in our Hacker News dataset. We'll use a set to check for both Python with a capital 'P' and python with a lowercase 'p'.

In [16]:
# import module re
import re 

# extract a list 'titles' containing all the titles from dataset
titles = hn['title'].tolist()

python_mentions = 0

pattern = '[Pp]ython'

for title in titles:
    if re.search(pattern, title):
        python_mentions += 1

print(python_mentions)

160


### Counting Matches with pandas Methods

In [24]:
titles = hn['title']
pattern = '[Pp]ython'

python_mentions = titles.str.contains(pattern).sum()

print(python_mentions)

160


### Using Regular Expressions to Select Data

In [29]:
titles = hn['title']

ruby_titles = titles[titles.str.contains('[Rr]uby')]

print(ruby_titles.head())

190                    Ruby on Google AppEngine Goes Beta
484          Related: Pure Ruby Relational Algebra Engine
1388    Show HN: HTTPalooza  Ruby's greatest HTTP clie...
1949    Rewriting a Ruby C Extension in Rust: How a Na...
2022    Show HN: CrashBreak  Reproduce exceptions as f...
Name: title, dtype: object


### Quantifiers

In [33]:
pattern = 'e-?mail'
email_bool = titles.str.contains(pattern)
email_count = email_bool.sum()
email_titles = titles[email_bool]

print(email_count)
print('\n')
print(email_titles.head())

86


119     Show HN: Send an email from your shell to your...
313         Disposable emails for safe spam free shopping
1361    Ask HN: Doing cold emails? helps us prove this...
1750    Protect yourself from spam, bots and phishing ...
2421                   Ashley Madison hack treating email
Name: title, dtype: object


### Character Classes 